# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment, Dataset
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core import Environment
from azureml.core import ScriptRunConfig
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform,choice
import os
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.train.automl import AutoMLConfig
import pandas as pd
from sklearn import model_selection


## Dataset

### Overview
The dataset to be used is built up from meteorological values for the city of Los Angeles, USA. It includes data like the minimum and maximum temperature on the day, the wind speed and the measured precipitation. 
The objective of this project will be to make a model capable of predicting the precipitation in the city depending on the other given meteorogical measures. For this we will use the autoML tool from Azure as well as a predifined regression model with optimized hyperparameters using HyperDrive.

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'Capstone_ML'

experiment=Experiment(ws, experiment_name)

found = False
key = "Precipitation-LA"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 
        

if not found:
        # Gets a dataset from a given URL
        example_data = 'https://raw.githubusercontent.com/RaikohGrass/ML_Azure_Capstone/main/precipitation_LA.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        df = dataset.to_pandas_dataframe()
        df.describe()
        # We drop the empty column TAVG
        dataset = dataset.drop_columns('TAVG')
        # Since we are not using deep learning or techniques that have a notion of time we can drop the date and PGTM columns
        # We also drop the station name column
        dataset = dataset.drop_columns(['STATION','NAME','DATE','PGTM'])
        #Register the dataset in the default datastore
        dataset = dataset.register(workspace=ws,
                                   name=key)




In [3]:
# Lets look at the dataframe
df.describe()

,AWND,PRCP,TMAX,TMIN,WDF2,WDF5,WSF2,WSF5,WT01,WT02,WT08
count,1823.000000,1827.000000,1827.000000,1827.000000,1822.000000,1809.000000,1823.000000,1809.000000,551.0,39.0,861.0
mean,1.675710,0.035720,76.498632,58.120416,254.143798,256.428966,9.062095,14.721669,1.0,1.0,1.0
std,0.909434,0.189859,9.133929,7.223756,54.756284,55.571902,2.016827,3.622257,0.0,0.0,0.0
min,0.000000,0.000000,52.000000,38.000000,10.000000,10.000000,2.900000,6.000000,1.0,1.0,1.0
25%,1.120000,0.000000,70.000000,53.000000,260.000000,270.000000,8.100000,12.100000,1.0,1.0,1.0
50%,1.570000,0.000000,76.000000,58.000000,270.000000,270.000000,8.900000,14.100000,1.0,1.0,1.0
75%,2.010000,0.000000,83.000000,64.000000,270.000000,280.000000,10.100000,16.100000,1.0,1.0,1.0
max,8.050000,2.670000,111.000000,79.000000,360.000000,360.000000,21.000000,34.000000,1.0,1.0,1.0


In [4]:
df.head()

,STATION,NAME,DATE,AWND,PGTM,PRCP,TAVG,TMAX,TMIN,WDF2,WDF5,WSF2,WSF5,WT01,WT02,WT08
0,USW00093134,"LOS ANGELES DOWNTOWN USC, CA US",2016-01-01,2.46,None,0.00,None,64,43,10.0,30.0,8.1,11.0,NaN,NaN,1.0
1,USW00093134,"LOS ANGELES DOWNTOWN USC, CA US",2016-01-02,2.01,None,0.00,None,65,47,270.0,30.0,6.0,8.9,NaN,NaN,NaN
2,USW00093134,"LOS ANGELES DOWNTOWN USC, CA US",2016-01-03,0.67,None,0.00,None,62,44,150.0,150.0,10.1,14.1,NaN,NaN,NaN
3,USW00093134,"LOS ANGELES DOWNTOWN USC, CA US",2016-01-04,1.34,None,0.01,None,69,55,270.0,280.0,8.1,14.1,NaN,NaN,NaN
4,USW00093134,"LOS ANGELES DOWNTOWN USC, CA US",2016-01-05,2.46,None,1.61,None,59,49,140.0,140.0,10.1,16.1,1.0,1.0,NaN


### Creating the compute target
Here we will define the compute cluster for our runs

In [5]:
cpu_cluster_name = 'EduardoCluster'
compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',max_nodes=4)
cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [9]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'normalized_root_mean_squared_error'
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target='EduardoCluster',
                             task = "regression",
                             training_data=dataset,
                             label_column_name="PRCP", 
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [10]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
Capstone_ML,AutoML_18db5fca-6e86-473e-b5f6-eca05c360738,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [11]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

We can retrieve the best model from the autoML run



In [12]:
best_autoML_run, best_autmoML_model = remote_run.get_output()

In [49]:
best_autoML_run

Experiment,Id,Type,Status,Details Page,Docs Page
Capstone_ML,AutoML_18db5fca-6e86-473e-b5f6-eca05c360738_52,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [52]:
best_autmoML_model

RegressionPipeline(pipeline=Pipeline(memory=None,
                                     steps=[('datatransformer',
                                             DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='regression', working_dir='/mnt/batch/ta...
)), ('extratreesregressor', ExtraTreesRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mse', max_depth=None, max_features=0.8, max_leaf_nodes=None, max_samples=None, min_impurity_decrease=0.0, min_impurity_split=None, min_samples_leaf=0.003466237459044996, min_samples_split=0.026024633095283807, min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1, oob_score=False, random_state=None, verbose=0, warm_start=False))], verbose=False))], weights=[0.13333333333333333, 0.06666666666666667, 0.26666666666666666, 0.3333333333333333, 0.06666666666666

In [13]:
best_autoML_run.get_metrics()

{'median_absolute_error': 0.0018805139769623145,
 'normalized_root_mean_squared_error': 0.051396387642265534,
 'normalized_mean_absolute_error': 0.014277990315210551,
 'spearman_correlation': 0.3876555200585674,
 'root_mean_squared_log_error': nan,
 'mean_absolute_percentage_error': 136.9448068295247,
 'explained_variance': 0.45568347054361086,
 'root_mean_squared_error': 0.13722835500484898,
 'r2_score': 0.4556454966898967,
 'normalized_root_mean_squared_log_error': nan,
 'normalized_median_absolute_error': 0.0007043123509222152,
 'mean_absolute_error': 0.03812223414161217,
 'predicted_true': 'aml://artifactId/ExperimentRun/dcid.AutoML_18db5fca-6e86-473e-b5f6-eca05c360738_52/predicted_true',
 'residuals': 'aml://artifactId/ExperimentRun/dcid.AutoML_18db5fca-6e86-473e-b5f6-eca05c360738_52/residuals'}

In [14]:
best_autoML_run.get_details()

{'runId': 'AutoML_18db5fca-6e86-473e-b5f6-eca05c360738_52',
 'target': 'EduardoCluster',
 'status': 'Completed',
 'startTimeUtc': '2022-01-27T15:17:26.876725Z',
 'endTimeUtc': '2022-01-27T15:18:50.287916Z',
 'services': {},
 'properties': {'runTemplate': 'automl_child',
  'pipeline_id': '__AutoML_Ensemble__',
  'pipeline_spec': '{"pipeline_id":"__AutoML_Ensemble__","objects":[{"module":"azureml.train.automl.ensemble","class_name":"Ensemble","spec_class":"sklearn","param_args":[],"param_kwargs":{"automl_settings":"{\'task_type\':\'regression\',\'primary_metric\':\'normalized_root_mean_squared_error\',\'verbosity\':20,\'ensemble_iterations\':15,\'is_timeseries\':False,\'name\':\'Capstone_ML\',\'compute_target\':\'EduardoCluster\',\'subscription_id\':\'9b72f9e6-56c5-4c16-991b-19c652994860\',\'region\':\'southcentralus\',\'spark_service\':None}","ensemble_run_id":"AutoML_18db5fca-6e86-473e-b5f6-eca05c360738_52","experiment_name":"Capstone_ML","workspace_name":"quick-starts-ws-179145","subs

From the properties we can see that the best model was a Voting Ensemble using the following algorithms:

- 2'ExtremeRandomTrees', 
- 2 'XGBoostRegressor'
- 'RandomForest
- 'DecisionTree'

We then proceed to register the model for later deployment

In [15]:
#TODO: Save the best model
autoML_model = best_autoML_run.register_model(model_name = 'best_autoML_model', model_path = './outputs/model.pkl')

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [16]:
from azureml.core import Model
from azureml.core.conda_dependencies import CondaDependencies

environment = best_autoML_run.get_environment()

from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice


# we retrieve the score script generated with the registered model
save_path = 'score.py'
best_autoML_run.download_file('outputs/scoring_file_v_1_0_0.py', save_path)

inference_config = InferenceConfig(entry_script = save_path, environment=environment)
aci_config = AciWebservice.deploy_configuration(cpu_cores=2, memory_gb=2,description = 'precipitation predicter for LA')

service_name = 'precipitation-predictor'
service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[autoML_model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True)


In [17]:
service.wait_for_deployment(show_output = True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-01-27 15:24:32+00:00 Creating Container Registry if not exists..
2022-01-27 15:34:33+00:00 Registering the environment.
2022-01-27 15:34:33+00:00 Use the existing image.
2022-01-27 15:34:34+00:00 Generating deployment configuration..
2022-01-27 15:34:35+00:00 Submitting deployment to compute.
2022-01-27 15:34:38+00:00 Checking the status of deployment precipitation-predictor..
2022-01-27 15:35:14+00:00 Checking the status of inference endpoint precipitation-predictor.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [18]:
print('Service REST URI: {}'.format(service.scoring_uri))
print('Swagger URI: {}'.format(service.swagger_uri))

Service REST URI: http://242f6566-ca52-4fcf-ae14-3a08667665ff.southcentralus.azurecontainer.io/score
Swagger URI: http://242f6566-ca52-4fcf-ae14-3a08667665ff.southcentralus.azurecontainer.io/swagger.json


TODO: In the cell below, send a request to the web service you deployed to test it.

In [44]:
import requests
import urllib.request
import json

# We get a sample from the original dataframe
df = dataset.to_pandas_dataframe()
data_sample = df.sample(5).fillna(0)
precipitation_sample = data_sample.pop('PRCP')

# Transform the data sample into a json format
dataload = json.dumps({'data': data_sample.to_dict(orient='records')})
dataload = str.encode(json.dumps({'data': data_sample.to_dict(orient='records')}))
headers = {'Content-type': 'application/json'}


# Send a post request to the service
#dataload = '{"data": [{"AWND": 5.59,"TMAX": 71,"TMIN": 53,"WDF2": 310.00,"WDF5": 310.00,"WSF2": 16.10,"WSF5": 25.90,"WT01": 0.00,"WT02": 0.00,"WT08": 0.00}]}'
#post_request = requests.post(service.scoring_uri,dataload,headers)
#post_request.text

post_request = urllib.request.Request(service.scoring_uri, dataload, headers)

response = urllib.request.urlopen(post_request)
result = response.read()
print(result)


b'"{\\"result\\": [0.06685696427728961, -0.0001924298416333471, 0.009576353641844441, -0.0002360099351743369, 1.2924192980797615]}"'


In [45]:
dataload

b'{"data": [{"AWND": 5.59, "TMAX": 77, "TMIN": 60, "WDF2": 290.0, "WDF5": 290.0, "WSF2": 14.1, "WSF5": 27.1, "WT01": 0.0, "WT02": 0.0, "WT08": 0.0}, {"AWND": 1.12, "TMAX": 90, "TMIN": 68, "WDF2": 270.0, "WDF5": 270.0, "WSF2": 8.1, "WSF5": 13.0, "WT01": 0.0, "WT02": 0.0, "WT08": 1.0}, {"AWND": 1.34, "TMAX": 80, "TMIN": 53, "WDF2": 280.0, "WDF5": 280.0, "WSF2": 8.9, "WSF5": 15.0, "WT01": 1.0, "WT02": 0.0, "WT08": 0.0}, {"AWND": 1.12, "TMAX": 75, "TMIN": 60, "WDF2": 260.0, "WDF5": 270.0, "WSF2": 6.9, "WSF5": 12.1, "WT01": 0.0, "WT02": 0.0, "WT08": 1.0}, {"AWND": 7.16, "TMAX": 61, "TMIN": 53, "WDF2": 130.0, "WDF5": 140.0, "WSF2": 17.0, "WSF5": 30.0, "WT01": 1.0, "WT02": 0.0, "WT08": 0.0}]}'

In [46]:
precipitation_sample

480    0.00
946    0.00
411    0.00
1406   0.00
413    2.01
Name: PRCP, dtype: float64

TODO: In the cell below, print the logs of the web service and delete the service

In [48]:
# Printing the logs of the service
print(service.get_logs())

2022-01-27T15:35:03,011709000+00:00 - rsyslog/run 
2022-01-27T15:35:03,013029100+00:00 - gunicorn/run 
2022-01-27T15:35:03,010880100+00:00 - iot-server/run 
Dynamic Python package installation is disabled.
Starting HTTP server
rsyslogd: /azureml-envs/azureml_248925b58f9a398a4b0c980c8749f7a2/lib/libuuid.so.1: no version information available (required by rsyslogd)
2022-01-27T15:35:03,056272600+00:00 - nginx/run 
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2022-01-27T15:35:03,285568800+00:00 - iot-server/finish 1 0
2022-01-27T15:35:03,290081100+00:00 - Exit code 1 is normal. Not restarting iot-server.
Starting gunicorn 20.1.0
Listening at: http://127.0.0.1:31311 (82)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 111
SPARK_HOME not set. Skipping PySpark Initialization.
Generating new fontManager, this may take some time...
Initializing logger
2022-01-27 15:35:05,494 | root | INFO | Starting up app insights client
logging socket was

In [54]:
# save environment and dependencies of the model

save_path = 'autoML_environment_specifications.yml'
best_autoML_run.download_file('outputs/conda_env_v_1_0_0.yml', save_path)

save_path = 'autoML_environment_dependencies.json'
best_autoML_run.download_file('outputs/env_dependencies.json', save_path)


In [ ]:
# Deleting the service
service.delete()


**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
